In [8]:
using DiffusionMLE, Plots, PyCall, CSV, DataFrames, HDF5
plotlyjs()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17578080976854643674\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-5618586922994318007\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17578080976854643674\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

# Introduction

This is a minimal example on how to apply our maximum likelihood estimator to a set of homogeneous single-particle tracking data.  Here, "homogeneous" refers to the fact that the underlying diffusive dynamics is the same for all trajectories.  

For more details on the theoretical framework, please refer to the associated preprint:
> J. T. Bullerjahn and G. Hummer, "Maximum likelihood estimates of diffusion coefficients from single-molecule tracking experiments", https://arxiv.org/abs/2011.09955

# Generate trajectories

Each trajectory can be seen as a $d$-dimensional array (<code>Array{Float64,2}</code>), so the data set should be of the type <code>Array{Array{Float64,2},1}</code>.  

Here, we generate mock data, made up of $M$ $d$-dimensional trajectories of different lengths $N = \{N_{1}, N_{2}, \dots, N_{M}\}$.  The $N_{i}$ are distributed uniformly on the interval $[3,100]$.  

In [9]:
include("smeared_trajectory_integrator.jl")

const M = 2 # Number of trajectories
const d = 2 # Dimension of trajectories

const N_sub = 100 # Number of substeps over which the trajectory is smeared out

N = [ rand(3:5) for i = 1 : M ] # Array of trajectory lengths

const a2 = 1.2
const σ2 = 2.3

B = [1/6 for m = 1 : M] # Array of blurring coefficients, where we have assumed a uniform illumination profile
data = make_2D_data(N,N_sub,a2,σ2); # Mock data set

In [10]:
# Import some test trajectories...
tester = raw"C:\Users\User\OneDrive\Documents\Python Programs\Piezo1_MLE\Trajectory_CSVs\test_data.csv"
test_data = CSV.read(tester, DataFrame)
trackIDs = unique(test_data, :ID).ID

3-element Vector{Int64}:
 0
 1
 2

In [11]:
function load_traj_csv(raw_csv_file_path)
    csv_data = CSV.read(raw_csv_file_path, DataFrame)
    csv_data = dropmissing(csv_data)
    trackIDs = unique(csv_data, :ID).ID
    trk_count = length(trackIDs)
    B_values = [1/6 for m = 1 : trk_count]
    sing_cond_tracks = Array{Array{Float64,2},1}(undef, trk_count)
    n = 0
    for i in trackIDs
        n += 1
        indiv_track = csv_data[in([i]).(csv_data.ID), :]
        sing_cond_tracks[n] = Array(indiv_track[:, [:X, :Y]])
    end
    return sing_cond_tracks, B_values
end

load_traj_csv (generic function with 1 method)

In [12]:
try1, B_values = load_traj_csv(tester)

([[159.86739365022228 344.5599053020969; 160.37170742438224 344.74759384162735; … ; 159.52969883557083 344.3255584771516; 159.5987459186825 344.5406183222827], [172.29367468129357 197.1746257186286; 172.25453001814677 197.05318975199054; … ; 172.24074501999928 197.1851131585509; 172.33875339943808 197.41858239102532], [178.90877090947671 25.94517091061468; 178.7293084516537 25.659295139056315; … ; 178.94168438264728 26.121333271410684; 179.1371790995336 26.053483993397805]], [0.16666666666666666, 0.16666666666666666, 0.16666666666666666])

# Analyzing the data

### Parameter estimation

In [13]:
parameters = MLE_estimator(B_values,try1)
uncertainties = MLE_errors(B_values,try1,parameters)

println("Estimates:")
println(string("a2 = ", parameters[1], " ± ", uncertainties[1]))
println(string("σ2 = ",parameters[2], " ± ", uncertainties[2]))

println()

println("Ground truth:")
println(string("a2 = ", a2))
println(string("σ2 = ", σ2))

Estimates:
a2 = 0.013512456492704526 ± 0.013486593008823074
σ2 = 0.052948323857680946 ± 0.02468520785232314

Ground truth:
a2 = 1.2
σ2 = 2.3


### Diffusion coefficients

Diffusion coefficients can be extracted from the $\sigma^2$-values, irrespective of the dimension $d$, as follows:
$\begin{equation*}
D = \frac{\sigma^2}{2 \Delta t} \, .  
\end{equation*}$
Here, $\sigma$ has the same dimension as the data, i.e., if the trajectories are recorded on the nanometer scale then $[\sigma] = \textrm{nm}$, and $\Delta t$ denotes the time step between two observations.  

# Testing model assumptions

We have assumed that the data are homogeneous.  To test this and other assumptions of our diffusion model (Gaussianity, structure of covariance matrix, etc.), we perform a quality factor analysis:

In [14]:
Q = Q_factor_analysis(B_values,try1,parameters)

println("Kuiper statistic:")
println(string("κ = ", Kuiper_statistic!(Q)))

Plots.histogram(Q, normed=true, legend=false,
    xlabel="quality factor Q",
    ylabel="probability distribution function p(Q)",
    title="Survival of the Fittest")

Kuiper statistic:
κ = 1.2550765574879794


<!DOCTYPE html>
 
 
 Plots.jl

The uniform distribution of the quality factors, as well as the fact that the Kuiper statistic evaluates close to unity, indicate that the assumptions underlying the model are satisfied.  This is, of course, not very surprising, because the mock data was generated with said assumptions in mind.  